In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
filepath = '/kaggle/input/icr-identify-age-related-conditions'
df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
sub = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

# Encoding
df_train['EJ'] = df_train['EJ'].replace({'A': 0, 'B': 1})
df_test['EJ']  = df_test['EJ'].replace({'A': 0, 'B': 1})

# Özellikler (X) ve etiketler (y) ayrımı
X = df_train.drop('Class', axis=1)
y = df_train['Class']

In [3]:
df_test.fillna(df_test.mean(), inplace=True)
df_test.isnull().sum().sort_values(ascending=False).head(60)

AB     0
AF     0
DL     0
DN     0
DU     0
DV     0
DY     0
EB     0
EE     0
EG     0
EH     0
EJ     0
EL     0
EP     0
EU     0
FC     0
FD     0
FE     0
FI     0
FL     0
FR     0
FS     0
GB     0
GE     0
GF     0
GH     0
GI     0
DI     0
DH     0
DF     0
BR     0
AH     0
AM     0
AR     0
AX     0
AY     0
AZ     0
BC     0
BD     0
BN     0
BP     0
BQ     0
BZ     0
DE     0
CB     0
CC     0
CD     0
CF     0
CH     0
CL     0
CR     0
CS     0
CU     0
CW     0
DA     0
GL     0
dtype: int64

In [4]:
# Sütun adları listesi
sutunlar = ['EL', 'BQ', 'CC', 'CB', 'FS', 'FL', 'FC', 'DU', 'GL']

# Eksik değerleri sütunların ortalamalarıyla doldurma
for sutun in sutunlar:
    sutun_ortalama = X[sutun].mean()
    X[sutun].fillna(sutun_ortalama, inplace=True)
    
X = X.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})
df_test = df_test.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})

### Threshold değerini kendim belirlediğim kod bloğu:
# deneme yaparak değiştirdim.

In [5]:
# Veri setinizin adı "X" olsun
# Threshold değerini burada belirleyin (örn: 1.5)
threshold = 4

# "X" veri setini yükleyin veya oluşturun

# Aykırı değerleri bulup ortalama ile değiştiren fonksiyon
def replace_outliers_with_mean(X, threshold):
    for column in X.columns:
        data = X[column]
        mean = data.mean()
        std = data.std()
        lower_bound = mean - threshold * std
        upper_bound = mean + threshold * std
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        X.loc[outliers.index, column] = mean
    return X

# Aykırı değerleri ortalama ile doldurun
X = replace_outliers_with_mean(X, threshold)

# Sonuçları görselleştirme ve kontrol etme
# print("\nAykırı Değerleri Ortalama ile Doldurulmuş Veri Seti:")
# print(X.head())


# VERİ NORMALİZASYONU VE STANDARDİZASYONU

In [6]:
# Her sütunun ortalamasını ve standart sapmasını hesaplayın
mean_values = np.mean(X, axis=0)
std_values = np.std(X, axis=0)

# Standardizasyon işlemi
X_standardized = (X - mean_values) / std_values

# Normalizasyon işlemi (min-max scaling)
X_normalized = (X_standardized - np.min(X_standardized, axis=0)) / (np.max(X_standardized, axis=0) - np.min(X_standardized, axis=0))

# BURAYI BEN EKLEDİM, HATA VERİRSE DÜZELT!
X = X_normalized

# X_normalized veri setiniz, hem standardize edilmiş hem de normalize edilmiş haliyle kullanılabilir

# 2 ŞEY DENİYORUM

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb

# Veri setini eğitim ve test verileri olarak ayırın (örneğin, %80 eğitim, %20 test olarak ayıralım)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)

# Sınıf ağırlıklandırmasını belirleyin
class_weights = {0: 0.18, 1: 0.82}

# Yarışma metriğim
def balanced_log_loss(y_true, y_pred):

    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1) / 2

# XGB MODELİ
#lambda : 0.0000263894617720096 idi.
#alpha : 0.000463768723479341 idi.
#gamma : 0.000847289463422307 idi.

xgb_params = {
    'n_estimators': 700,
    'learning_rate': 0.01,
    'booster': 'gbtree',
    'lambda': 6,
    'alpha': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_depth': 3,
    'min_child_weight': 3,
    'gamma': 0.1,
    'n_jobs': -1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'verbosity': 0,
    'random_state': 42,
    'scale_pos_weight': 0.90 / 0.10,  # class_weights
}

# XGBoost Model
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict_proba(X_test)[:, 1]

xgb_auc = roc_auc_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, np.round(y_pred_xgb))
xgb_acc = accuracy_score(y_test, np.round(y_pred_xgb))
xgb_prec = precision_score(y_test, np.round(y_pred_xgb))
xgb_recall = recall_score(y_test, np.round(y_pred_xgb))
xgb_balanced_log_loss = balanced_log_loss(y_test, y_pred_xgb)

print("XGBoost Model ROC AUC Score:", xgb_auc)
print("XGBoost Model F1 Score:", xgb_f1)
print("XGBoost Model Accuracy:", xgb_acc)
print("XGBoost Model Precision:", xgb_prec)
print("XGBoost Model Recall:", xgb_recall)
print("XGBoost Model Balanced Log Loss:", xgb_balanced_log_loss)
print("------------------------------------------------------------------------------")

XGBoost Model ROC AUC Score: 0.9854636591478697
XGBoost Model F1 Score: 0.8500000000000001
XGBoost Model Accuracy: 0.9516129032258065
XGBoost Model Precision: 0.8095238095238095
XGBoost Model Recall: 0.8947368421052632
XGBoost Model Balanced Log Loss: 0.22568298375168652
------------------------------------------------------------------------------


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# GRADIENT COP OLDUGU ICIN BURAYA ALMAYA GEREK BILE DUYMADIM. HE ILERIDE BIR GUN GEREKIRSE ALIRIM.

# Veri setini bölmek için Stratified K-Fold kullanalım
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Modellerin performans sonuçlarını saklamak için boş listeler oluşturalım
xgb_roc_auc_scores, xgb_f1_scores, xgb_acc_scores, xgb_prec_scores, xgb_recall_scores, xgb_balanced_log_loss_scores = [], [], [], [], [], []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # XGBoost Model
    xgb_model = xgb.XGBClassifier(**xgb_params)
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict_proba(X_test)[:, 1]

    xgb_auc = roc_auc_score(y_test, y_pred_xgb)
    xgb_f1 = f1_score(y_test, np.round(y_pred_xgb))
    xgb_acc = accuracy_score(y_test, np.round(y_pred_xgb))
    xgb_prec = precision_score(y_test, np.round(y_pred_xgb))
    xgb_recall = recall_score(y_test, np.round(y_pred_xgb))
    xgb_balanced_log_loss = balanced_log_loss(y_test, y_pred_xgb)

    xgb_roc_auc_scores.append(xgb_auc)
    xgb_f1_scores.append(xgb_f1)
    xgb_acc_scores.append(xgb_acc)
    xgb_prec_scores.append(xgb_prec)
    xgb_recall_scores.append(xgb_recall)
    xgb_balanced_log_loss_scores.append(xgb_balanced_log_loss)
    
# Ortalama değerleri saklamak için boş listeler oluşturun
xgb_mean_roc_auc = np.mean(xgb_roc_auc_scores)
xgb_mean_f1 = np.mean(xgb_f1_scores)
xgb_mean_acc = np.mean(xgb_acc_scores)
xgb_mean_prec = np.mean(xgb_prec_scores)
xgb_mean_recall = np.mean(xgb_recall_scores)
xgb_mean_balanced_log_loss = np.mean(xgb_balanced_log_loss_scores)

# Sonuçları yazdırın
print("XGBoost Model Mean ROC AUC:", xgb_mean_roc_auc)
print("XGBoost Model Mean F1 Score:", xgb_mean_f1)
print("XGBoost Model Mean Accuracy:", xgb_mean_acc)
print("XGBoost Model Mean Precision:", xgb_mean_prec)
print("XGBoost Model Mean Recall:", xgb_mean_recall)
print("XGBoost Model Mean Balanced Log Loss:", xgb_mean_balanced_log_loss)


XGBoost Model Mean ROC AUC: 0.9569996915657952
XGBoost Model Mean F1 Score: 0.7594543904518329
XGBoost Model Mean Accuracy: 0.9043010752688172
XGBoost Model Mean Precision: 0.6896387520525451
XGBoost Model Mean Recall: 0.8510822510822511
XGBoost Model Mean Balanced Log Loss: 0.2860677880405552


# LIGHT GBM DENEMESİ

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
import lightgbm as lgb

# Veri setini eğitim ve test verileri olarak ayırın (örneğin, %80 eğitim, %20 test olarak ayıralım)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)

# Sınıf ağırlıklandırmasını belirleyin
class_weights = {0: 0.18, 1: 0.82}

# Yarışma metriğim
def balanced_log_loss(y_true, y_pred):
    
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1) / 2

# LightGBM MODELİ
#lambda : 0.0000263894617720096 idi.
#alpha : 0.000463768723479341 idi.
#gamma : 0.000847289463422307 idi.

lgb_params = {
    'boosting_type': 'gbdt', #gbdt idi
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.075, # en iyi değer 0.1 ya da 0.075 dir.
    'lambda_l2': 8, # Best 8 'di.
    'alpha': 1, # etki etmeyipti.
    'subsample': 0.8, # etki etmeyipti.
    'colsample_bytree': 0.7, # Best 7'di.
    'max_depth': 3, # 3 Bestti.
    'min_child_samples': 3, #3 bestti.
    'min_split_gain': 0.1, # 0.1 Bestti.
    'num_leaves' : 5,
    'n_jobs': -1,
    'verbose': -1,
    'scale_pos_weight': 0.90 / 0.10,  # class_weights ilginç şekilde 90 a 10 bestti.
    'random_state': 42,
}

# LightGBM Model
lgb_model = lgb.LGBMClassifier(**lgb_params)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict_proba(X_test)[:, 1]

lgb_auc = roc_auc_score(y_test, y_pred_lgb)
lgb_f1 = f1_score(y_test, np.round(y_pred_lgb))
lgb_acc = accuracy_score(y_test, np.round(y_pred_lgb))
lgb_prec = precision_score(y_test, np.round(y_pred_lgb))
lgb_recall = recall_score(y_test, np.round(y_pred_lgb))
lgb_balanced_log_loss = balanced_log_loss(y_test, y_pred_lgb)

print("LightGBM Model ROC AUC Score:", lgb_auc)
print("LightGBM Model F1 Score:", lgb_f1)
print("LightGBM Model Accuracy:", lgb_acc)
print("LightGBM Model Precision:", lgb_prec)
print("LightGBM Model Recall:", lgb_recall)
print("LightGBM Model Balanced Log Loss:", lgb_balanced_log_loss)
print("------------------------------------------------------------------------------")

[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
LightGBM Model ROC AUC Score: 0.9804511278195489
LightGBM Model F1 Score: 0.7804878048780488
LightGBM Model Accuracy: 0.9274193548387096
LightGBM Model Precision: 0.7272727272727273
LightGBM Model Recall: 0.8421052631578947
LightGBM Model Balanced Log Loss: 0.2367526451286298
------------------------------------------------------------------------------


In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
import numpy as np
import lightgbm as lgb

# Veri setini bölmek için Stratified K-Fold kullanalım
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=53)

# Modellerin performans sonuçlarını saklamak için boş listeler oluşturalım
lgb_roc_auc_scores, lgb_f1_scores, lgb_acc_scores, lgb_prec_scores, lgb_recall_scores, lgb_balanced_log_loss_scores = [], [], [], [], [], []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # LightGBM Model
    lgb_model = lgb.LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train, y_train)
    y_pred_lgb = lgb_model.predict_proba(X_test)[:, 1]

    lgb_auc = roc_auc_score(y_test, y_pred_lgb)
    lgb_f1 = f1_score(y_test, np.round(y_pred_lgb))
    lgb_acc = accuracy_score(y_test, np.round(y_pred_lgb))
    lgb_prec = precision_score(y_test, np.round(y_pred_lgb))
    lgb_recall = recall_score(y_test, np.round(y_pred_lgb))
    lgb_balanced_log_loss = balanced_log_loss(y_test, y_pred_lgb)

    lgb_roc_auc_scores.append(lgb_auc)
    lgb_f1_scores.append(lgb_f1)
    lgb_acc_scores.append(lgb_acc)
    lgb_prec_scores.append(lgb_prec)
    lgb_recall_scores.append(lgb_recall)
    lgb_balanced_log_loss_scores.append(lgb_balanced_log_loss)
    
# Ortalama değerleri saklamak için boş listeler oluşturun
lgb_mean_roc_auc = np.mean(lgb_roc_auc_scores)
lgb_mean_f1 = np.mean(lgb_f1_scores)
lgb_mean_acc = np.mean(lgb_acc_scores)
lgb_mean_prec = np.mean(lgb_prec_scores)
lgb_mean_recall = np.mean(lgb_recall_scores)
lgb_mean_balanced_log_loss = np.mean(lgb_balanced_log_loss_scores)

# Sonuçları yazdırın
print("LightGBM Model Mean ROC AUC:", lgb_mean_roc_auc)
print("LightGBM Model Mean F1 Score:", lgb_mean_f1)
print("LightGBM Model Mean Accuracy:", lgb_mean_acc)
print("LightGBM Model Mean Precision:", lgb_mean_prec)
print("LightGBM Model Mean Recall:", lgb_mean_recall)
print("LightGBM Model Mean Balanced Log Loss:", lgb_mean_balanced_log_loss)


[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
LightGBM Model Mean ROC AUC: 0.9609670210718552
LightGBM Model Mean F1 Score: 0.7796581989769903
LightGBM Model Mean Accuracy: 0.9108313663781799
LightGBM Model Mean Precision: 0.6900555064348167
LightGBM Model Mean Recall: 0.8978354978354979
LightGBM Model Mean Balanced Log Loss: 0.26383255919892773


# Submission

In [11]:
# En iyi parametreleri kullanarak modeli tüm veri setiyle tekrar eğitin
best_model = lgb.LGBMClassifier(**lgb_params)
best_model.fit(X, y)

# Test verileri üzerinde tahmin yapın (olasılık değerlerini alın)
test_probabilities = best_model.predict_proba(df_test)

# 'sub' veri çerçevesini güncelleme
sub['class_0'] = test_probabilities[:, 0]
sub['class_1'] = test_probabilities[:, 1]

# Sadece 'Id', 'class_0' ve 'class_1' sütunlarını içerecek şekilde yeni bir DataFrame oluşturun
submission_df = sub[['Id', 'class_0', 'class_1']]

# Submission dosyasını kaydetme
submission_df.to_csv("submission.csv", index=False)

# Oluşturulan submission_df'yi yazdırma
submission_df


[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8


,Id,class_0,class_1
0,00eed32682bb,0.51852,0.48148
1,010ebe33f668,0.51852,0.48148
2,02fa521e1838,0.51852,0.48148
3,040e15f562a2,0.51852,0.48148
4,046e85c7cc7f,0.51852,0.48148
